In [1]:
import pandas as pd

In [2]:
df = pd.read_pickle('data.pkl')

In [3]:
df = df[[
    'date_block_num',
    'shop_id',
    'item_id',
    'item_cnt_month',
    'zip_code',
    'item_category_id',
    'category_type',
    'category_device',
    'item_cnt_month_lag_1',
    'item_cnt_month_lag_2',
    'item_cnt_month_lag_3',
    'item_cnt_month_lag_6',
    'item_cnt_month_lag_12',
    'date_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_2',
    'date_item_avg_item_cnt_lag_3',
    'date_item_avg_item_cnt_lag_6',
    'date_item_avg_item_cnt_lag_12',
    'date_shop_avg_item_cnt_lag_1',
    'date_shop_avg_item_cnt_lag_2',
    'date_shop_avg_item_cnt_lag_3',
    'date_shop_avg_item_cnt_lag_6',
    'date_shop_avg_item_cnt_lag_12',
    'date_cat_avg_item_cnt_lag_1',
    'date_shop_cat_avg_item_cnt_lag_1',
    'date_city_avg_item_cnt_lag_1',
    'date_item_city_avg_item_cnt_lag_1',
    'delta_price_trend_with_lag_1',
    'month',
    'days_in_month',
    'item_last_sold_in_given_shop',
    'item_last_sold',
    'item_first_sold_in_given_shop',
    'item_first_sold'
]]

In [4]:
df_og = pd.read_pickle('data_og.pkl')

df_og = df_og[[
    'date_block_num',
    'shop_id',
    'item_id',
    'item_cnt_month',
    'city_code',
    'item_category_id',
    'type_code',
    'subtype_code',
    'item_cnt_month_lag_1',
    'item_cnt_month_lag_2',
    'item_cnt_month_lag_3',
    'item_cnt_month_lag_6',
    'item_cnt_month_lag_12',
    'date_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_2',
    'date_item_avg_item_cnt_lag_3',
    'date_item_avg_item_cnt_lag_6',
    'date_item_avg_item_cnt_lag_12',
    'date_shop_avg_item_cnt_lag_1',
    'date_shop_avg_item_cnt_lag_2',
    'date_shop_avg_item_cnt_lag_3',
    'date_shop_avg_item_cnt_lag_6',
    'date_shop_avg_item_cnt_lag_12',
    'date_cat_avg_item_cnt_lag_1',
    'date_shop_cat_avg_item_cnt_lag_1',
    'date_city_avg_item_cnt_lag_1',
    'date_item_city_avg_item_cnt_lag_1',
    'delta_price_lag',
    'month',
    'days',
    'item_shop_last_sale',
    'item_last_sale',
    'item_shop_first_sale',
    'item_first_sale',
]]

In [5]:
# sort his values that are sorted up to the month 34
df = df.sort_values(['date_block_num', 'shop_id', 'item_id'])
df_og = df_og.sort_values(['date_block_num', 'shop_id', 'item_id'])
df = df.reset_index(drop=True)
df_og = df_og.reset_index(drop=True)

In [6]:
# methodological differences
df['delta_price_trend_with_lag_1'] = df_og['delta_price_lag']
df['item_last_sold_in_given_shop'] = df_og['item_shop_last_sale']
df['item_last_sold'] = df_og['item_last_sale']
df['item_first_sold_in_given_shop'] = df_og['item_shop_first_sale']
df['item_first_sold'] = df_og['item_first_sale']

# different encodings
df['zip_code'] = df_og['city_code']
df['category_type'] = df_og['type_code']
df['category_device'] = df_og['subtype_code']
df['month'] = df_og['month']

# max 0.002% diff, sometimes even 0.0001%
df['date_avg_item_cnt_lag_1'] = df_og['date_avg_item_cnt_lag_1']
df['date_shop_cat_avg_item_cnt_lag_1'] = df_og['date_shop_cat_avg_item_cnt_lag_1']
df['date_city_avg_item_cnt_lag_1'] = df_og['date_city_avg_item_cnt_lag_1']
df['date_cat_avg_item_cnt_lag_1'] = df_og['date_cat_avg_item_cnt_lag_1']
df['date_shop_avg_item_cnt_lag_1'] = df_og['date_shop_avg_item_cnt_lag_1']
df['date_shop_avg_item_cnt_lag_2'] = df_og['date_shop_avg_item_cnt_lag_2']
df['date_shop_avg_item_cnt_lag_3'] = df_og['date_shop_avg_item_cnt_lag_3']
df['date_shop_avg_item_cnt_lag_6'] = df_og['date_shop_avg_item_cnt_lag_6']
df['date_shop_avg_item_cnt_lag_12'] = df_og['date_shop_avg_item_cnt_lag_12']

# has actual discrepencies
df['date_item_city_avg_item_cnt_lag_1'] = df_og['date_item_city_avg_item_cnt_lag_1']

In [7]:
X_train = df_og[df_og.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = df_og[df_og.date_block_num < 33]['item_cnt_month']
X_valid = df_og[df_og.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = df_og[df_og.date_block_num == 33]['item_cnt_month']
X_test = df_og[df_og.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [8]:
df.loc[df.shop_id == 10, 'shop_id'] = 11

In [9]:
# df = df[df.date_block_num == 34]
# df = df[df.shop_id == 2]
#
# df_og = df_og[df_og.date_block_num == 34]
# df_og = df_og[df_og.shop_id == 2]
#
df.to_csv('test_full_mine.csv')
df_og.to_csv('test_full_his.csv')

In [10]:
# from xgboost import XGBRegressor
#
# model = XGBRegressor(
#     max_depth=8,
#     n_estimators=1000,
#     min_child_weight=300,
#     colsample_bytree=0.8,
#     subsample=0.8,
#     eta=0.3,
#     seed=42)
#
# model.fit(
#     X_train,
#     Y_train,
#     eval_metric="rmse",
#     eval_set=[(X_train, Y_train), (X_valid, Y_valid)],
#     verbose=True,
#     early_stopping_rounds = 10)
#
# test = pd.read_csv('data/technical/test.csv')
#
# Y_pred = model.predict(X_valid).clip(0, 20)
# Y_test = model.predict(X_test).clip(0, 20)
#
# submission = pd.DataFrame({
#     "ID": test.index,
#     "item_cnt_month": Y_test
# })
# submission.to_csv('xgb_submission.csv', index=False)